In [ ]:
import xarray as xr
import pandas as pd
from holoviews import opts
import holoviews as hv
import panel as pn
import numpy as np
import datetime as dt
from get_data import get_urls_mod,get_urls_obs, get_data
from basic_widgets import create_widgets, populate_vars_1D
from bokeh.models import HoverTool

In [ ]:
hv.extension('bokeh')
pn.extension()

# Use-case: Time Series

## Introduction

This file contains the code for generating a time series plot for one model for a specific site and time and variable as well as corresponding observation. (OBSERVATION TO BE ADDED WHEN THEY EXIST IN PORTAL)

The code for the functions for fetching data can be found in read_data.py

This plotting tool as been generated for the Year Of Polar Prediction site Model Intercomparison Project, YOPPsiteMIP

## Code

The following cell contains the functions responsible for plotting the data. 

This use-case has two scenarios, which are user-selectable through widgets.

- The first where all the model files for a time period are plotted in full, which leads to overlap between         their shared dates.

- The second is where only one day is selected per model (wheater it be the first,second or third day is user-selectable) and are concatinated into one non-overlapping time series. 

In [ ]:
def plot_ts_stacked(data_mod,data_obs,variable):
    plots = []
    
    for item in data_mod.columns.get_level_values(1):
        p = hv.Curve((data_mod.index,data_mod[variable][item])).opts(
            tools=[
                HoverTool(
                    tooltips=[
                        ("Label","Model"),
                        ("Start date",str(data_mod[variable][item].first_valid_index())),
                        ("End date", str(data_mod[variable][item].last_valid_index()))
                    ],
                    toggleable=False
                )
            ])
        plots.append(p)
    plot_obs = hv.Curve(data_obs, label= "Observation")
    plot_obs.opts(color = "black")
    plots.append(plot_obs)
    plot = hv.Overlay(plots)

    return plot

def plot_ts_concat(data_mod,data_obs,variable):
    plot_mod = hv.Curve((data_mod.index,data_mod[variable]),label = "Model")
    plot_obs = hv.Curve((data_obs),label = "Observation")
    plot_obs.opts(color = "black")
    plot = hv.Overlay([plot_mod,plot_obs])
    plot
    return plot

The code in the following cell is responsible for the widgets that enable user-selectability. 
TO DO: populate variables dynamically

In [ ]:
#widgets for user selection of model, plot type, site, variable, start time and date range
#most selections will be expanded in the near fututre to include all relevant variables,
#models and sites. 
#there will also be a selection of SOP1 and SOP2, currently the dates are only SOP1
types = pn.widgets.Select(
    name="types", options=['concatenated','stacked'],value='concatenated', margin=(0, 20, 0, 0)
)

#dictionary = populate_vars_1D()
#widgets = create_widgets(dictionary)

#site_name = widgets[0] 
site_name = pn.widgets.Select(
    name="Site", options=['tiksi','sodankyla'],value = 'tiksi', margin=(0, 20, 0, 0)
)

#model_name = widgets[1]
model_name = pn.widgets.Select(
    name="model", options=['slav-rhmc','ifs-ecmwf'], margin=(0, 20, 0, 0)
)

#user selection variable
variable = pn.widgets.Select(
    name="Variables", options=['tas','ts','meep'], margin=(0, 20, 0, 0)
)
#variable = widgets[2]

#dates = widgets[3]

#concat_day = widgets[4]

#start_time = widgets[5]

#user selection start and end date only sop 1 as of right now
dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 1), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 1), dt.datetime(2018, 2, 2))
)

#user selection start time for model 
start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

#user selection of which day should be fetched from the model files
#for concat
concat_day = pn.widgets.Select(
    name="Model day selected", options={1:0,2:1,3:2},value=3, margin=(0,20,0,0)
)

loading = pn.indicators.LoadingSpinner(value=False, width=100, height=100)

The following cell contains the function responsible for fetching the data and calling the plotting routines. 

In [ ]:
@pn.depends(types,site_name,model_name,variable,start_time,dates,concat_day)
def main_timeseries(types,site_name,model_name,variable,start_time,dates,concat_day):
    
    loading.value = True
    
    urls_mod = get_urls_mod(dates[0],dates[1],start_time,model_name,site_name,[variable],concat_day)
    url_obs = get_urls_obs(site_name,"timeSeries","1")

        
    data_mod,error_mod = get_data(types,urls_mod,concat_day,"model")
    data_obs, error_obs = get_data("observation",url_obs,concat_day,"observation")
    print(data_mod)

    if error_mod is not None:
        loading.value = False
        pane = pn.pane.Alert("Fetching model: "+error_mod)
        return pane
    elif error_obs is not None:
        loading.value = False
        return pn.pane.Alert("Fetching observations: "+error_obs)
    

    data_mod = data_mod.to_dataframe()  
    data_obs = data_obs[variable].to_dataframe()
 
    if types == 'concatenated':
        data_obs = data_obs[dates[0]:dates[1]+dt.timedelta(days=1)]
        plot = plot_ts_concat(data_mod,data_obs, variable)
        print("concat plot")   
    else:
        data_obs = data_obs[dates[0]:dates[1]+dt.timedelta(days=3)]
        plot = plot_ts_stacked(data_mod.unstack(),data_obs, variable)  
        print("stacked plot")

    plot.opts(title="Timevar "+variable+" "+model_name+" "+site_name,
          width=900,
          height=400,
          ylabel=variable,
          xlabel="time: "+dates[0].strftime("%Y-%-m-%d")+ " - " +dates[1].strftime("%Y-%-m-%d")
         )
        
    loading.value = False
   
    return plot

The follwing cell contains the function responsible for deploying the application

In [ ]:
pn.serve(pn.Column("timevar 1 sop 1", 
          main_timeseries,
          pn.Row(loading,pn.Column(site_name,model_name,variable),
                 pn.Column(types,start_time),pn.Column(dates,concat_day)),
          width_policy="max"))
